## Read and Save Query Status in CSV for all Workflows

<br>

**Read all the JSON files for all the workflows and print out the messages and query status to a CSV file**

<br>

In [84]:
# import all the modules. NB: submit_run_ars_modules contains all the modules to submit job to ARAX

import json
import requests
from gamma_viewer import GammaViewer
from IPython.display import display
#from submit_run_ars_modules import submit_to_ars, submit_to_devars, printjson, retrieve_devars_results
import glob 
import os
from collections import defaultdict
import pandas as pd
from time import sleep
from os import path
from datetime import datetime
import numpy as np



In [2]:
def flatten_list(_2d_list):
    flat_list = []
    # Iterate through the outer list
    for element in _2d_list:
        if type(element) is list:
            # If the element is of type list, iterate through the sublist
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list

In [90]:
def submit_to_ars(m,ars_url='https://ars.transltr.io/ars/api',arax_url='https://arax.ncats.io'):
    submit_url=f'{ars_url}/submit'
    response = requests.post(submit_url,json=m)
    try:
        message_id = response.json()['pk']
    except:
        print('fail')
        message_id = None
    print(f'{arax_url}/?source=ARS&id={message_id}')
    return message_id

##https://ars.ci.transltr.io/ars/api

def retrieve_ars_results(mid, name, check_sheet, ars_url='https://ars.transltr.io/ars/api'):
    pk = 'https://arax.ncats.io/?source=ARS&id=' + mid
    message_url = f'{ars_url}/messages/{mid}?trace=y'
    response = requests.get(message_url)
    j = response.json()
    print( j['status'] )
    results = {}
    dictionary = {}
    dictionary_2 = {}
    dict3 = {}
    for child in j['children']:
        print(child['status'])
        error_code = child['code']
        
        if child['status']  == 'Done':
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                nresults = len(child_response['fields']['data']['message']['results'])
                if nresults > 0:
                    results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
                    
                    if name in list(check_sheet.Workflow):
                        print(name)
                        dfy = check_sheet[check_sheet['Workflow']== name]

                        dfy.reset_index(drop=True)

                        for index,curie_id in enumerate(dfy.Curie):
                            print(index,curie_id)
                            node_num = dfy.iloc[index][3]
                            query_id = curie_id
                            if np.isnan(dfy.iloc[index][2]) == True:
                                len_check = len(child_response['fields']['data']['message']['results'])
                                len_check = int(len_check)
                            else:
                                len_check = int(dfy.iloc[index][2])

                            #print(node_num, query_id, len_check)

                            locs = []
                            for x, val in enumerate(child_response['fields']['data']['message']['results']):
                                #print(val)

                                if x < len_check:

                                    if query_id in val['node_bindings'][node_num][0]['id']:
                                        locs.append(x)
                            if not locs:
                                check_result = f'False'
                                print(check_result)
                                #pass
                            else:
                                check_result = f'True'
                                print('curie id:', query_id, ': INCLUDED at postion N ==', locs, 'on', node_num)

                            dict3[curie_id] = check_result    

                if child_response['fields']['data']['message']['knowledge_graph']['edges']:
                    if child_response['fields']['data']['message']['knowledge_graph']['edges'].keys():
                            edge_ex = child_response['fields']['data']['message']['knowledge_graph']['edges']
                            test_att_values =[]
                            for val in child_response['fields']['data']['message']['knowledge_graph']['edges'].keys():
                                #print(val)
                                
                                for tx in edge_ex[val]['attributes']:
                                    if (tx['attribute_type_id'] == 'biolink:primary_knowledge_source') or (tx['attribute_type_id'] == 'biolink:original_knowledge_source') or (tx['attribute_type_id'] == 'biolink:aggregator_knowledge_source') :
                                        
                                        
                                        value_att = tx['value']
                        
                                        test_att_values.append(value_att)
                                        test_att = set(flatten_list(test_att_values))
                                        
                                        
                                        dictionary_2[child['actor']['agent']] = test_att
                    #else:
                        #dictionary_2[child['actor']['agent']] = [] 
                #else:
                   # dictionary_2[child['actor']['agent']] = []
            
            except Exception as e:
                nresults=0
                child['status'] = 'ARS Error'
                #dictionary_2[child['actor']['agent']] = []
                
            
        
        elif child['status'] == 'Error':
            nresults=0
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
                #dictionary_2[child['actor']['agent']] = []
            except Exception as e:
                #print(e)
                child['status'] = 'ARS Error'
                #dictionary_2[child['actor']['agent']] = []
        
        
        else:
            nresults = 0
            #dictionary_2[child['actor']['agent']] = []
            
        dictionary['pk_id'] =  pk  
            
        if ((child['status'] == 'Done') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'No Results' ': ' + str(error_code)
            #test =  [child['actor']['agent'], 'No Results']
        elif ((child['status'] == 'ARS Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'ARS Error' ': ' + str(error_code)
        elif ((child['status'] == 'Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'Error' ': ' + str(error_code)
            #test =  [child['actor']['agent'], 'ARS Error']
        elif ((child['status'] == 'Done') & (nresults != 0)):
            #test =  [child['actor']['agent'], 'Results']
            dictionary[child['actor']['agent']] = 'Results' ': ' + str(error_code) + ' ' + str(dict3)
        elif ((child['status'] == 'Unknown') & (nresults == 0)):
            #test =  [child['actor']['agent'], 'Results']
            dictionary[child['actor']['agent']] = 'Unknown' ': ' + str(error_code)
        
        
        print(child['actor']['agent'], child['status'], nresults)
        #test =  [child['actor']['agent'], child['status'], nresults]
        #test2.append(test)
    return [dictionary, dictionary_2]


#def submit_to_devars(m):
#    return submit_to_ars(m,ars_url='https://ars-dev.transltr.io/ars/api',arax_url='https://arax.ncats.io')

#def retrieve_devars_results(m):
#     return retrieve_ars_results(m,ars_url='https://ars-dev.transltr.io/ars/api')

def printjson(j):
    print(json.dumps(j,indent=4))
    
def make_hyperlink(value):
    return '=HYPERLINK("%s", "%s")' % (value.format(value), value)

In [93]:
check_sheet = pd.read_csv("/Users/priyash/Documents/GitHub/minihackathons/Notebooks/Query results to include or exclude - Sheet1.csv", header=0)

In [94]:
check_sheet

,Workflow,Curie,N (size of list of results),Query node ID,Include/Exclude,Unnamed: 5,Unnamed: 6
0,A.0_RHOBTB2_direct.json,PUBCHEM.COMPOUND:2950270,10.0,n1,Include,NaN,Note: this example row means that for workflow...
1,B.1_DILI-three-hop-interacts-with.json,CHEBI:41879,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
2,B.1_DILI-three-hop-interacts-with.json,MESH:D000077185,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
3,B.1_DILI-three-hop-interacts-with.json,MESH:D000077385,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
4,B.1_DILI-three-hop-interacts-with.json,MESH:D003474,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
5,B.1_DILI-three-hop-interacts-with.json,RXNORM:40068,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
6,B.1_DILI-three-hop-interacts-with.json,PUBCHEM.COMPOUND:5877,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
7,B.1_DILI-three-hop-interacts-with.json,PUBCHEM.COMPOUND:5755,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
8,C.1a_SmallMolecule_real_world_evidence_MultScl...,MONDO:0005301,100.0,n00,include,NaN,NaN
9,C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallM...,CHEBI:45783,100.0,n3,include,NaN,NaN


<br>

**The below code reads each JSON files from the Workflows A through D (subdirectories). The queries are submitted to ARAX and output is saved in a dictionary, where the key is the file name of the JSON to denote which query is being run and the values assigned to the key is the query id**

<br>


In [6]:
PATH = r'/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo'
EXT = "*.json"
dict_workflows = {}
for root, dirs, files in os.walk(PATH): # step 1: accessing file
    #print(root)
    for name in files:
        
        if name.endswith((".json")):
            if (name == 'C.1_Template_SmallMolecule_real_world_evidence_Disease.json') or (name == 'C.2_Template_Drug_Disease_GeneSet_interacts_with_SmallMolecule.json') or (name == 'C.3_Template_Disease_related_to_Drug.json'):
                pass
            else:
                file_read = path.join(root, name)
                dir_name = (os.path.splitext(os.path.basename(root))[0])
                print(file_read)

                filename = (os.path.splitext(os.path.basename(file_read))[0])
                print(filename)
                with open(file_read,'r') as inf:
                    query = json.load(inf)

                    kcresult = submit_to_ars(query)

                    sleep(500)

                    result_status = retrieve_ars_results(kcresult, name, check_sheet)


                    dict_workflows[filename] = kcresult

                    sleep(100)

/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.3_KCNMA1.json
A.3_KCNMA1
https://arax.ncats.io/?source=ARS&id=3f5e652f-288e-4d13-8bf0-7ebb0b5a1291
Done
Done
kp-cohd ARS Error 0
Done
kp-textmining Done 0
Error
ara-aragorn-exp Error 0
Done
ara-unsecret Done 102
Done
kp-chp Done 30
Error
kp-icees Error 0
Unknown
ara-ncats Unknown 0
Done
kp-icees-dili Done 0
Done
ara-bte Done 18
Done
kp-openpredict Done 0
Done
ara-arax Done 172
Done
kp-molecular Done 99
Done
ara-improving Done 5
Done
ara-explanatory Done 0
Done
ara-aragorn Done 951
Done
kp-cam Done 0
Error
ara-robokop Error 0
Done
kp-genetics Done 0
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.2a_RHOBTB2_twohop.json
A.2a_RHOBTB2_twohop
https://arax.ncats.io/?source=ARS&id=1d34c24b-d5ed-4c85-a994-4435af070ee4
Done
Error
ara-aragorn-exp Error 0
Error
kp-molecular Error 0
Done
kp-chp Done 0
Done
ara-arax Done 0
Done
kp-cam Done 0
Done
ara-aragorn Done 0
Done
ara-bte Done 0
Done
ara-improving 

https://arax.ncats.io/?source=ARS&id=ee932c26-cb53-4e1c-914a-2de00a0854c6
Done
Done
ara-aragorn Done 2562
Error
ara-aragorn-exp Error 0
Done
kp-chp Done 0
Done
kp-icees Done 0
Done
ara-arax Done 178
Error
ara-robokop Error 0
Unknown
ara-ncats Unknown 0
Done
kp-cam Done 0
Done
kp-molecular Done 0
Done
kp-cohd ARS Error 0
Done
kp-genetics Done 0
Error
ara-unsecret Error 0
Done
kp-textmining Done 0
Done
ara-improving Done 16
Done
kp-icees-dili Done 0
Done
ara-explanatory Done 0
Done
ara-bte Done 305
Done
kp-openpredict Done 0
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowB/B.2e_DILI-fourth-one-hop-from-RXNORM_40068_DextranSulfate.json
B.2e_DILI-fourth-one-hop-from-RXNORM_40068_DextranSulfate
https://arax.ncats.io/?source=ARS&id=179eee32-9cfb-4e79-8235-fdd7ae1da0b6
Done
Done
kp-cam Done 0
Error
kp-molecular Error 0
Error
ara-arax Error 0
Error
ara-aragorn-exp Error 0
Error
kp-genetics Error 0
Done
kp-textmining Done 0
Error
kp-openpredict Error 0
Done
ara-unsecret Don

<br>

### Codes below are for recording messages and generating outout as csv

<br>

In [7]:
dict_workflows

{'A.3_KCNMA1': '3f5e652f-288e-4d13-8bf0-7ebb0b5a1291',
 'A.2a_RHOBTB2_twohop': '1d34c24b-d5ed-4c85-a994-4435af070ee4',
 'A.8_EGFR_simple': 'a15dc206-d88d-48e4-83db-5f82ea2e5626',
 'A.Multiomics_BigGIM_DR_KP_RHOBTB2': 'e2e377c1-fb0b-4ddc-9cf9-5644213b2977',
 'A.0_RHOBTB2_direct': 'bf599739-3fbc-4e24-8df7-376ab5228ec1',
 'A.2_RHOBTB2_twohop': 'd83d5052-574d-485d-980e-9bbc264fe4fe',
 'A.Multiomics_BigGIM_DR_KP_EGFR': 'fd99fbea-15be-40f4-9f70-09997d4ec5de',
 'A.2_RHOBTB2_twohop_constrained': 'e93dda7a-772b-46df-8f23-0e527656c1b4',
 'A.9_EGFR_advanced': '6bde158d-b793-410c-a6fd-0a59563c4f1f',
 'A.2a_expanded_RHOBTB2_twohop_constrained': 'e368d144-31ef-4625-857d-06e42f378501',
 'A.1_RHOBTB2': '04834ad0-a9e6-44f6-a9c0-a9a2ca5c0f68',
 'A.2a_expanded_RHOBTB2_twohop': 'a309911a-a915-40f7-881d-d84d1e130461',
 'B.2b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol': 'ee932c26-cb53-4e1c-914a-2de00a0854c6',
 'B.2e_DILI-fourth-one-hop-from-RXNORM_40068_DextranSulfate': '179eee32-9cfb-4e79-8235-fd

In [100]:
workflow_result_messages = {}
for keys, val in dict_workflows.items():
    name = keys + '.json'
    print(name, val)
    

    result_status = retrieve_ars_results(val, name, check_sheet)

    workflow_result_messages[keys] = result_status

A.3_KCNMA1.json 3f5e652f-288e-4d13-8bf0-7ebb0b5a1291
Done
Done
kp-cohd ARS Error 0
Done
kp-textmining Done 0
Error
ara-aragorn-exp Error 0
Done
ara-unsecret Done 102
Done
kp-chp Done 30
Error
kp-icees Error 0
Unknown
ara-ncats Unknown 0
Done
kp-icees-dili Done 0
Done
ara-bte Done 18
Done
kp-openpredict Done 0
Done
ara-arax Done 172
Done
kp-molecular Done 99
Done
ara-improving Done 5
Done
ara-explanatory Done 0
Done
ara-aragorn Done 951
Done
kp-cam Done 0
Error
ara-robokop Error 0
Done
kp-genetics Done 0
A.2a_RHOBTB2_twohop.json 1d34c24b-d5ed-4c85-a994-4435af070ee4
Done
Error
ara-aragorn-exp Error 0
Error
kp-molecular Error 0
Done
kp-chp Done 0
Done
ara-arax Done 0
Done
kp-cam Done 0
Done
ara-aragorn Done 0
Done
ara-bte Done 0
Done
ara-improving ARS Error 0
Error
ara-robokop Error 0
Done
ara-unsecret Done 0
Done
kp-textmining Done 0
Done
ara-explanatory Done 0
Error
kp-cohd Error 0
Unknown
ara-ncats Unknown 0
Done
kp-icees-dili ARS Error 0
Unknown
kp-genetics Unknown 0
Error
kp-icees Er

kp-textmining Done 0
Error
ara-aragorn-exp Error 0
Done
ara-unsecret Done 0
Done
kp-molecular Done 0
B.2a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone.json 18b974c8-860d-490a-97b3-82531d512bda
Done
Done
ara-arax Done 500
Done
kp-genetics Done 0
Unknown
ara-ncats Unknown 0
Done
ara-improving Done 219
Done
ara-bte Done 2817
Done
ara-unsecret Done 347
Error
ara-aragorn Error 0
Done
kp-textmining Done 0
Done
kp-chp Done 0
Done
kp-cohd Done 50
Done
kp-openpredict Done 300
Done
kp-molecular Done 242
Done
kp-icees-dili Done 0
Error
ara-aragorn-exp Error 0
Done
ara-explanatory Done 0
Done
kp-icees Done 0
Done
kp-cam Done 0
Error
ara-robokop Error 0
B.2d_DILI-fourth-one-hop-from-MESH_D003474_Curcumin.json af01aa4d-8481-4ed7-b608-1484f75ec481
Done
Done
kp-genetics Done 0
Done
ara-aragorn Done 2624
Error
ara-robokop Error 0
Done
ara-arax Done 500
Unknown
ara-ncats Unknown 0
Done
kp-textmining Done 0
Error
ara-aragorn-exp Error 0
Done
kp-cohd ARS Error 0
Done
kp-molecular Done 0
Done
kp-open

kp-icees-dili ARS Error 0
Done
kp-icees ARS Error 0
Error
kp-cohd Error 0
Error
ara-aragorn Error 0
Done
kp-chp Done 0
C.2b_Etanercept_MultSclerosis_GeneSet_and_SmallMolecule.json 85ef5639-4f7e-4129-8572-47309cfa815e
Done
Error
ara-robokop Error 0
Unknown
kp-genetics Unknown 0
Error
ara-improving Error 0
Done
kp-icees-dili ARS Error 0
Error
kp-molecular Error 0
Done
ara-explanatory Done 0
Done
C.2b_Etanercept_MultSclerosis_GeneSet_and_SmallMolecule.json
0 CHEBI:4875
False
ara-aragorn Done 347
Error
kp-cohd Error 0
Done
kp-cam Done 0
Error
ara-bte Error 0
Unknown
ara-ncats Unknown 0
Done
kp-chp Done 0
Done
kp-openpredict Done 0
Done
kp-textmining Done 0
Error
ara-aragorn-exp Error 0
Done
ara-unsecret Done 0
Done
C.2b_Etanercept_MultSclerosis_GeneSet_and_SmallMolecule.json
0 CHEBI:4875
False
ara-arax Done 100
Done
kp-icees ARS Error 0
C.1a_SmallMolecule_real_world_evidence_MultSclerosis.json 272f2d8e-e0b1-42de-b87c-5c365b3fa9c3
Done
Done
C.1a_SmallMolecule_real_world_evidence_MultScleros

### Creating dataframe for workflows with PK

<br>

In [101]:
## Convert mesages to a dataframe
col = []
final_dict = defaultdict(list)


for k in sorted(workflow_result_messages):
    print(k)
    
    col.append(k)
    
    
    
    #count = 0
    for key, value in workflow_result_messages[k][0].items():
        
        
        #count= count+1
        final_dict[key].append(value)

    final_dict = dict(final_dict)
    #print(count)
df = pd.DataFrame(final_dict).T
df.rename(columns=dict(zip(df.columns, col)), inplace=True)

A.0_RHOBTB2_direct
A.1_RHOBTB2
A.2_RHOBTB2_twohop
A.2_RHOBTB2_twohop_constrained
A.2a_RHOBTB2_twohop
A.2a_expanded_RHOBTB2_twohop
A.2a_expanded_RHOBTB2_twohop_constrained
A.3_KCNMA1
A.8_EGFR_simple
A.9_EGFR_advanced
A.Multiomics_BigGIM_DR_KP_EGFR
A.Multiomics_BigGIM_DR_KP_RHOBTB2
B.1_DILI-three-hop-interacts-with
B.2a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone
B.2b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol
B.2c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin
B.2d_DILI-fourth-one-hop-from-MESH_D003474_Curcumin
B.2e_DILI-fourth-one-hop-from-RXNORM_40068_DextranSulfate
B.2f_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone
B.2g_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5865_Prednisone
B.3_DILI_branched-four-hop
C.1a_SmallMolecule_real_world_evidence_MultSclerosis
C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule
C.2b_Etanercept_MultSclerosis_GeneSet_and_SmallMolecule
C.2c_Natalizumab_MultSclerosis_GeneSet_and_SmallMolecule
C.3a_MultSclerosis_related_t

In [102]:
df

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,...,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule,C.2b_Etanercept_MultSclerosis_GeneSet_and_SmallMolecule,C.2c_Natalizumab_MultSclerosis_GeneSet_and_SmallMolecule,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
pk_id,https://arax.ncats.io/?source=ARS&id=bf599739-...,https://arax.ncats.io/?source=ARS&id=04834ad0-...,https://arax.ncats.io/?source=ARS&id=d83d5052-...,https://arax.ncats.io/?source=ARS&id=e93dda7a-...,https://arax.ncats.io/?source=ARS&id=1d34c24b-...,https://arax.ncats.io/?source=ARS&id=a309911a-...,https://arax.ncats.io/?source=ARS&id=e368d144-...,https://arax.ncats.io/?source=ARS&id=3f5e652f-...,https://arax.ncats.io/?source=ARS&id=a15dc206-...,https://arax.ncats.io/?source=ARS&id=6bde158d-...,...,https://arax.ncats.io/?source=ARS&id=272f2d8e-...,https://arax.ncats.io/?source=ARS&id=4ca87e5e-...,https://arax.ncats.io/?source=ARS&id=85ef5639-...,https://arax.ncats.io/?source=ARS&id=6920da18-...,https://arax.ncats.io/?source=ARS&id=e29b6b26-...,https://arax.ncats.io/?source=ARS&id=b5e37dbf-...,https://arax.ncats.io/?source=ARS&id=53e2b591-...,https://arax.ncats.io/?source=ARS&id=0a5306ab-...,https://arax.ncats.io/?source=ARS&id=577fed92-...,https://arax.ncats.io/?source=ARS&id=2da6777a-...
kp-textmining,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-explanatory,ARS Error: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-unsecret,Results: 200 {'PUBCHEM.COMPOUND:2950270': 'Fal...,No Results: 200,Results: 200 {},No Results: 200,No Results: 200,No Results: 200,No Results: 200,Results: 200 {},Results: 200 {},Results: 200 {},...,Results: 200 {'MONDO:0005301': 'True'},No Results: 200,No Results: 200,No Results: 200,Error: 598,Error: 598,Error: 598,Results: 200 {},No Results: 200,No Results: 200
kp-molecular,Results: 200 {'PUBCHEM.COMPOUND:2950270': 'Fal...,No Results: 200,Error: 501,Error: 501,Error: 501,Error: 501,Error: 501,Results: 200 {},Results: 200 {},Results: 200 {},...,Results: 200 {'MONDO:0005301': 'True'},Error: 501,Error: 501,Error: 501,No Results: 200,Error: 501,Error: 501,No Results: 200,Error: 501,Error: 501
kp-icees,Error: 598,No Results: 200,Error: 598,ARS Error: 200,Error: 598,ARS Error: 200,ARS Error: 200,Error: 598,Error: 598,No Results: 200,...,No Results: 200,ARS Error: 200,ARS Error: 200,ARS Error: 200,No Results: 200,ARS Error: 200,ARS Error: 200,No Results: 200,ARS Error: 200,ARS Error: 200
kp-icees-dili,No Results: 200,No Results: 200,ARS Error: 200,ARS Error: 200,ARS Error: 200,ARS Error: 200,ARS Error: 200,No Results: 200,No Results: 200,No Results: 200,...,No Results: 200,ARS Error: 200,ARS Error: 200,ARS Error: 200,No Results: 200,ARS Error: 200,ARS Error: 200,No Results: 200,ARS Error: 200,ARS Error: 200
kp-openpredict,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-ncats,Unknown: 503,Unknown

In [103]:
df.replace('ARS Error', 'No Results', regex=True,inplace=True)

In [104]:
df

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,...,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule,C.2b_Etanercept_MultSclerosis_GeneSet_and_SmallMolecule,C.2c_Natalizumab_MultSclerosis_GeneSet_and_SmallMolecule,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
pk_id,https://arax.ncats.io/?source=ARS&id=bf599739-...,https://arax.ncats.io/?source=ARS&id=04834ad0-...,https://arax.ncats.io/?source=ARS&id=d83d5052-...,https://arax.ncats.io/?source=ARS&id=e93dda7a-...,https://arax.ncats.io/?source=ARS&id=1d34c24b-...,https://arax.ncats.io/?source=ARS&id=a309911a-...,https://arax.ncats.io/?source=ARS&id=e368d144-...,https://arax.ncats.io/?source=ARS&id=3f5e652f-...,https://arax.ncats.io/?source=ARS&id=a15dc206-...,https://arax.ncats.io/?source=ARS&id=6bde158d-...,...,https://arax.ncats.io/?source=ARS&id=272f2d8e-...,https://arax.ncats.io/?source=ARS&id=4ca87e5e-...,https://arax.ncats.io/?source=ARS&id=85ef5639-...,https://arax.ncats.io/?source=ARS&id=6920da18-...,https://arax.ncats.io/?source=ARS&id=e29b6b26-...,https://arax.ncats.io/?source=ARS&id=b5e37dbf-...,https://arax.ncats.io/?source=ARS&id=53e2b591-...,https://arax.ncats.io/?source=ARS&id=0a5306ab-...,https://arax.ncats.io/?source=ARS&id=577fed92-...,https://arax.ncats.io/?source=ARS&id=2da6777a-...
kp-textmining,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-explanatory,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-unsecret,Results: 200 {'PUBCHEM.COMPOUND:2950270': 'Fal...,No Results: 200,Results: 200 {},No Results: 200,No Results: 200,No Results: 200,No Results: 200,Results: 200 {},Results: 200 {},Results: 200 {},...,Results: 200 {'MONDO:0005301': 'True'},No Results: 200,No Results: 200,No Results: 200,Error: 598,Error: 598,Error: 598,Results: 200 {},No Results: 200,No Results: 200
kp-molecular,Results: 200 {'PUBCHEM.COMPOUND:2950270': 'Fal...,No Results: 200,Error: 501,Error: 501,Error: 501,Error: 501,Error: 501,Results: 200 {},Results: 200 {},Results: 200 {},...,Results: 200 {'MONDO:0005301': 'True'},Error: 501,Error: 501,Error: 501,No Results: 200,Error: 501,Error: 501,No Results: 200,Error: 501,Error: 501
kp-icees,Error: 598,No Results: 200,Error: 598,No Results: 200,Error: 598,No Results: 200,No Results: 200,Error: 598,Error: 598,No Results: 200,...,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
kp-icees-dili,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
kp-openpredict,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-nca

In [105]:
df.replace('{}','',regex=True,inplace=True)

### Creating second table with edge attribute source

<br>

In [106]:
final_dict2 = defaultdict(dict)
for k in sorted(workflow_result_messages):
    print(k)
    col.append(k)
    
    count = 0
    
    for key, value in workflow_result_messages[k][1].items():
        final_dict2[k][key] = value

A.0_RHOBTB2_direct
A.1_RHOBTB2
A.2_RHOBTB2_twohop
A.2_RHOBTB2_twohop_constrained
A.2a_RHOBTB2_twohop
A.2a_expanded_RHOBTB2_twohop
A.2a_expanded_RHOBTB2_twohop_constrained
A.3_KCNMA1
A.8_EGFR_simple
A.9_EGFR_advanced
A.Multiomics_BigGIM_DR_KP_EGFR
A.Multiomics_BigGIM_DR_KP_RHOBTB2
B.1_DILI-three-hop-interacts-with
B.2a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone
B.2b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol
B.2c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin
B.2d_DILI-fourth-one-hop-from-MESH_D003474_Curcumin
B.2e_DILI-fourth-one-hop-from-RXNORM_40068_DextranSulfate
B.2f_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone
B.2g_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5865_Prednisone
B.3_DILI_branched-four-hop
C.1a_SmallMolecule_real_world_evidence_MultSclerosis
C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule
C.2b_Etanercept_MultSclerosis_GeneSet_and_SmallMolecule
C.2c_Natalizumab_MultSclerosis_GeneSet_and_SmallMolecule
C.3a_MultSclerosis_related_t

In [107]:
final_dict2

defaultdict(dict,
            {'A.0_RHOBTB2_direct': {'ara-unsecret': {'infores:ctd',
               'infores:molepro'},
              'kp-molecular': {'infores:ctd', 'infores:molepro'},
              'ara-arax': {'infores:arax', 'infores:ctd', 'infores:molepro'},
              'ara-improving': {'infores:improving-agent',
               'infores:lincs',
               'infores:spoke'}},
             'A.1_RHOBTB2': {'ara-improving': {'infores:improving-agent',
               'infores:lincs',
               'infores:spoke'},
              'ara-aragorn': {'infores:aragorn',
               'infores:aragorn-ranker-ara',
               'infores:automat-robokop',
               'infores:biothings-semmeddb-gene',
               'infores:hetio',
               'infores:lincs',
               'infores:rtx-kg2',
               'infores:semmeddb',
               'infores:spoke',
               'infores:sri-reference-kg',
               'infores:translator-biothings-explorer'},
              'ara-a

In [108]:
final_dictassemble = []
for k, vs in final_dict2.items():
    #print(k,vs)
    for kv, v in vs.items():
        for t in v:
            final_dictassemble.append([k,kv,t])

In [109]:
final_dictassemble

[['A.0_RHOBTB2_direct', 'ara-unsecret', 'infores:ctd'],
 ['A.0_RHOBTB2_direct', 'ara-unsecret', 'infores:molepro'],
 ['A.0_RHOBTB2_direct', 'kp-molecular', 'infores:ctd'],
 ['A.0_RHOBTB2_direct', 'kp-molecular', 'infores:molepro'],
 ['A.0_RHOBTB2_direct', 'ara-arax', 'infores:arax'],
 ['A.0_RHOBTB2_direct', 'ara-arax', 'infores:ctd'],
 ['A.0_RHOBTB2_direct', 'ara-arax', 'infores:molepro'],
 ['A.0_RHOBTB2_direct', 'ara-improving', 'infores:spoke'],
 ['A.0_RHOBTB2_direct', 'ara-improving', 'infores:improving-agent'],
 ['A.0_RHOBTB2_direct', 'ara-improving', 'infores:lincs'],
 ['A.1_RHOBTB2', 'ara-improving', 'infores:spoke'],
 ['A.1_RHOBTB2', 'ara-improving', 'infores:improving-agent'],
 ['A.1_RHOBTB2', 'ara-improving', 'infores:lincs'],
 ['A.1_RHOBTB2', 'ara-aragorn', 'infores:semmeddb'],
 ['A.1_RHOBTB2', 'ara-aragorn', 'infores:automat-robokop'],
 ['A.1_RHOBTB2', 'ara-aragorn', 'infores:translator-biothings-explorer'],
 ['A.1_RHOBTB2', 'ara-aragorn', 'infores:hetio'],
 ['A.1_RHOBTB2', 

In [110]:
column_names = ['Workflow', 'ARS-KPs', 'Values']
df2 = pd.DataFrame(final_dictassemble, columns=column_names)
df2 = df2.astype(str)

In [111]:
df2.Values = df2.Values.apply(lambda x: x[2:-2] if ('[' in x) else x)

In [112]:
df2test = df2.groupby(['Workflow','Values'])['ARS-KPs'].agg(list)

In [113]:
df2test = pd.DataFrame(df2test.unstack().T)

In [114]:
df2test

Workflow,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,A.Multiomics_BigGIM_DR_KP_EGFR,...,B.2g_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5865_Prednisone,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule,C.2b_Etanercept_MultSclerosis_GeneSet_and_SmallMolecule,C.2c_Natalizumab_MultSclerosis_GeneSet_and_SmallMolecule,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine
Values,,,,,,,,,,,,,,,,,,,,,
CTD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,[ara-aragorn],NaN,NaN,NaN
DisGeNET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,[ara-aragorn],NaN,NaN,NaN
DrugCentral Indication,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,[ara-aragorn],NaN,NaN,NaN
JensenLab Knowledge GHR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,[ara-aragorn],NaN,NaN,NaN
JensenLab Knowledge UniProtKB-KW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,[ara-aragorn],NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
infores:sri-reference-kg,NaN,[ara-aragorn],NaN,NaN,[ara-aragorn],NaN,[ara-aragorn],[ara-aragorn],NaN,[ara-aragorn],...,NaN,NaN,NaN,[ara-aragorn],NaN,NaN,[ara-aragorn],NaN,"[ara-bte, ara-aragorn]",NaN
infores:stitch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[ara-arax]
infores:tcrd,NaN,NaN,NaN,NaN,[ara-aragorn],NaN,NaN,"[ara-aragorn, ara-unsecret, ara-arax, kp-molec...","[kp-molecular, ara-arax, ara-unsecret]",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [115]:
#df2test.drop([''], axis=0, inplace=True)

In [116]:
#df2test.index = df2test.index.map(lambda x: x[2:-2] if ('[' in x) else x)

In [117]:
df2test

Workflow,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,A.Multiomics_BigGIM_DR_KP_EGFR,...,B.2g_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5865_Prednisone,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule,C.2b_Etanercept_MultSclerosis_GeneSet_and_SmallMolecule,C.2c_Natalizumab_MultSclerosis_GeneSet_and_SmallMolecule,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine
Values,,,,,,,,,,,,,,,,,,,,,
CTD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,[ara-aragorn],NaN,NaN,NaN
DisGeNET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,[ara-aragorn],NaN,NaN,NaN
DrugCentral Indication,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,[ara-aragorn],NaN,NaN,NaN
JensenLab Knowledge GHR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,[ara-aragorn],NaN,NaN,NaN
JensenLab Knowledge UniProtKB-KW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,[ara-aragorn],NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
infores:sri-reference-kg,NaN,[ara-aragorn],NaN,NaN,[ara-aragorn],NaN,[ara-aragorn],[ara-aragorn],NaN,[ara-aragorn],...,NaN,NaN,NaN,[ara-aragorn],NaN,NaN,[ara-aragorn],NaN,"[ara-bte, ara-aragorn]",NaN
infores:stitch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[ara-arax]
infores:tcrd,NaN,NaN,NaN,NaN,[ara-aragorn],NaN,NaN,"[ara-aragorn, ara-unsecret, ara-arax, kp-molec...","[kp-molecular, ara-arax, ara-unsecret]",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [118]:
#df2.replace([], 'None', regex=True,inplace=True)

In [119]:
#df2test = df2test.mask(df2test.applymap(type).eq(list) & ~df2test.astype(bool))

In [120]:
df2test = df2test.rename_axis(None)

In [121]:
df2test.columns.name = None

In [122]:
df2test.fillna('', inplace=True)

In [123]:
df2test

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,A.Multiomics_BigGIM_DR_KP_EGFR,...,B.2g_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5865_Prednisone,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule,C.2b_Etanercept_MultSclerosis_GeneSet_and_SmallMolecule,C.2c_Natalizumab_MultSclerosis_GeneSet_and_SmallMolecule,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine
CTD,,,,,,,,,,,...,,,,,,,[ara-aragorn],,,
DisGeNET,,,,,,,,,,,...,,,,,,,[ara-aragorn],,,
DrugCentral Indication,,,,,,,,,,,...,,,,,,,[ara-aragorn],,,
JensenLab Knowledge GHR,,,,,,,,,,,...,,,,,,,[ara-aragorn],,,
JensenLab Knowledge UniProtKB-KW,,,,,,,,,,,...,,,,,,,[ara-aragorn],,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
infores:sri-reference-kg,,[ara-aragorn],,,[ara-aragorn],,[ara-aragorn],[ara-aragorn],,[ara-aragorn],...,,,,[ara-aragorn],,,[ara-aragorn],,"[ara-bte, ara-aragorn]",
infores:stitch,,,,,,,,,,,...,,,,,,,,,,[ara-arax]
infores:tcrd,,,,,[ara-aragorn],,,"[ara-aragorn, ara-unsecret, ara-arax, kp-molec...","[kp-molecular, ara-arax, ara-unsecret]",,...,,,,,,,,,,
infores:translator-biothings-explorer,,"[ara-aragorn, ara-bte]","[ara-bte, ara-arax]","[ara-arax, ara-bte]","[ara-arax, ara-bte, ara-aragorn]","[ara-arax, ara-bte]","[ara-bte, ara-arax]",,"[ara-arax, ara-bte]","[ara-bte, ara-arax, ara-aragorn]",...,[ara-bte],[ara-bte],[ara-aragorn],,[ara-bte],,,,[ara-bte],


## Sorting The index to push infores ID's that are compliant to Column L and M in INFORES Catalog


<br>
<br>

In [124]:
infores_catalog = pd.read_csv("/Users/priyash/Documents/GitHub/minihackathons/Notebooks/InfoRes Catalog - Translator InfoRes Catalog.csv", header=1)

In [125]:
infores_catalog = infores_catalog[['id', 'name','translator category','has contributor']]

In [126]:
infores_catalog = infores_catalog[:335]

In [127]:
dict_map = {}
for i in df2test.index.values:
    if i in infores_catalog['id'].values:
        indices = infores_catalog[infores_catalog['id']==i].index[0]
        if pd.notnull(infores_catalog.iloc[indices]['has contributor']):
            dict_map[i] = infores_catalog.iloc[indices]['translator category']
        else:
            dict_map[i] = 'External Source'
    else:
        dict_map[i] = 'Illegal value'

In [128]:
dict_map

{'CTD': 'Illegal value',
 'DisGeNET': 'Illegal value',
 'DrugCentral Indication': 'Illegal value',
 'JensenLab Knowledge GHR': 'Illegal value',
 'JensenLab Knowledge UniProtKB-KW': 'Illegal value',
 'JensenLab Text Mining': 'Illegal value',
 'Monarch': 'Illegal value',
 'UniProt Disease': 'Illegal value',
 'icees-asthma': 'Illegal value',
 'icees-dili': 'Illegal value',
 'infores:aragorn': 'ARA',
 'infores:aragorn-ranker-ara': 'Illegal value',
 'infores:arax': 'ARA',
 'infores:automat-biolink': 'KP',
 'infores:automat-covidkop': 'KP',
 'infores:automat-ctd': 'KP',
 'infores:automat-drug-central': 'KP',
 'infores:automat-gtopdb': 'KP',
 'infores:automat-hetio': 'KP',
 'infores:automat-hmdb': 'KP',
 'infores:automat-pharos': 'KP',
 'infores:automat-robokop': 'KP',
 'infores:automat-uberongraph': 'KP',
 'infores:bgee': 'External Source',
 'infores:bindingdb': 'External Source',
 'infores:biothings-dgidb': 'KP',
 'infores:biothings-multiomics-biggim-drugresponse': 'KP',
 'infores:biothings

In [129]:
df2test['Translator_Category_Complaint_to_ColL&M_InforesCatalog']=df2test.index.map(dict_map)

In [130]:
df2test

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,A.Multiomics_BigGIM_DR_KP_EGFR,...,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule,C.2b_Etanercept_MultSclerosis_GeneSet_and_SmallMolecule,C.2c_Natalizumab_MultSclerosis_GeneSet_and_SmallMolecule,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,Translator_Category_Complaint_to_ColL&M_InforesCatalog
CTD,,,,,,,,,,,...,,,,,,[ara-aragorn],,,,Illegal value
DisGeNET,,,,,,,,,,,...,,,,,,[ara-aragorn],,,,Illegal value
DrugCentral Indication,,,,,,,,,,,...,,,,,,[ara-aragorn],,,,Illegal value
JensenLab Knowledge GHR,,,,,,,,,,,...,,,,,,[ara-aragorn],,,,Illegal value
JensenLab Knowledge UniProtKB-KW,,,,,,,,,,,...,,,,,,[ara-aragorn],,,,Illegal value
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
infores:sri-reference-kg,,[ara-aragorn],,,[ara-aragorn],,[ara-aragorn],[ara-aragorn],,[ara-aragorn],...,,,[ara-aragorn],,,[ara-aragorn],,"[ara-bte, ara-aragorn]",,KP
infores:stitch,,,,,,,,,,,...,,,,,,,,,[ara-arax],External Source
infores:tcrd,,,,,[ara-aragorn],,,"[ara-aragorn, ara-unsecret, ara-arax, kp-molec...","[kp-molecular, ara-arax, ara-unsecret]",,...,,,,,,,,,,External Source
infores:translator-biothings-explorer,,"[ara-aragorn, ara-bte]","[ara-bte, ara-arax]","[ara-arax, ara-bte]","[ara-arax, ara-bte, ara-aragorn]","[ara-arax, ara-bte]","[ara-bte, ara-arax]",,"[ara-arax, ara-bte]","[ara-bte, ara-arax, ara-aragorn]",...,[ara-bte],[ara-aragorn],,[ara-bte],,,,[ara-bte],,ARA


In [131]:
df2test['sort']=pd.Categorical(df2test['Translator_Category_Complaint_to_ColL&M_InforesCatalog'], ["KP", "ARA",'External Source', 'Illegal Value'])
df2test =df2test.sort_values(['sort'])
df2test  = df2test.rename_axis(None)

In [132]:
df2test

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,A.Multiomics_BigGIM_DR_KP_EGFR,...,C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule,C.2b_Etanercept_MultSclerosis_GeneSet_and_SmallMolecule,C.2c_Natalizumab_MultSclerosis_GeneSet_and_SmallMolecule,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,Translator_Category_Complaint_to_ColL&M_InforesCatalog,sort
infores:spoke,[ara-improving],"[ara-improving, ara-aragorn]",[ara-improving],[ara-improving],[ara-aragorn],,[ara-improving],[ara-improving],[ara-improving],,...,,,,,"[ara-improving, ara-aragorn]",[ara-improving],[ara-improving],[ara-improving],KP,KP
infores:lincs,[ara-improving],"[ara-improving, ara-aragorn]","[ara-bte, ara-improving]","[ara-improving, ara-bte]",[ara-aragorn],,"[ara-improving, ara-aragorn]",[ara-improving],[ara-improving],,...,,,,,[ara-aragorn],[ara-improving],,,KP,KP
infores:molepro,"[ara-unsecret, kp-molecular, ara-arax]",,[ara-arax],[ara-arax],[ara-aragorn],,"[ara-unsecret, ara-arax, kp-molecular, ara-ara...","[ara-aragorn, ara-unsecret, ara-arax, kp-molec...","[kp-molecular, ara-arax, ara-unsecret]","[ara-unsecret, kp-molecular, ara-aragorn]",...,,[ara-aragorn],,,,,[ara-aragorn],[ara-arax],KP,KP
infores:mychem-info,,,[ara-arax],[ara-arax],,,,,,,...,[ara-aragorn],,,,,,[ara-bte],,KP,KP
infores:mydisease-info,,,,,,,,,,,...,,,[ara-bte],,,,[ara-bte],,KP,KP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
infores:aragorn-ranker-ara,,[ara-aragorn],,,[ara-aragorn],,[ara-aragorn],[ara-aragorn],,[ara-aragorn],...,[ara-aragorn],[ara-aragorn],,[ara-aragorn],[ara-aragorn],,[ara-aragorn],,Illegal value,NaN
infores:faers,,,,,,,[ara-aragorn],[ara-aragorn],,[ara-aragorn],...,,[ara-aragorn],,,,,"[ara-bte, ara-aragorn]",,Illegal value,NaN
infores:hetio,,[ara-aragorn],[ara-bte],[ara-bte],[ara-aragorn],,[ara-aragorn],[ara-aragorn],,[ara-aragorn],...,[ara-aragorn],[ara-aragorn],,,[ara-aragorn],,[ara-aragorn],,Illegal value,NaN
infores:ontological-hierarchy,,,,,[ara-aragorn],,[ara-aragorn],[ara-aragorn],,[ara-aragorn],...,,,,,[ara-aragorn],,[ara-aragorn],,Illegal value,NaN


In [133]:
cols = df2test.columns.tolist()
cols = [cols[-2]] + cols[:-2]
df2test = df2test[cols]

In [134]:
df2test['Query Type'] = 'Sync'

df2test = df2test[['Query Type']+ list(df2test.columns[:-1])]

<br>

## Converting the Pk's to hyperlink

<br>

In [135]:
df.loc['pk_id'] = df.loc['pk_id'].apply(lambda x: make_hyperlink(x))

In [136]:
df.rename({'pk_id': 'pk'}, inplace=True)

In [137]:
df.sort_index(inplace=True)

In [138]:
df

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,...,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule,C.2b_Etanercept_MultSclerosis_GeneSet_and_SmallMolecule,C.2c_Natalizumab_MultSclerosis_GeneSet_and_SmallMolecule,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
ara-aragorn,No Results: 200,Results: 200,Error: 598,No Results: 200,No Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,No Results: 200,...,Results: 200 {'MONDO:0005301': 'True'},Results: 200 {'CHEBI:45783': 'False'},Results: 200 {'CHEBI:4875': 'False'},Error: 598,Results: 200,"Results: 200 {'NCBIGene:120892': 'True', 'NCBI...",Error: 598,Results: 200,Error: 598,Error: 598
ara-aragorn-exp,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,...,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404
ara-arax,Results: 200 {'PUBCHEM.COMPOUND:2950270': 'Fal...,Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,...,Results: 200 {'MONDO:0005301': 'True'},Results: 200 {'CHEBI:45783': 'False'},Results: 200 {'CHEBI:4875': 'False'},Results: 200 {'CHEMBL.COMPOUND:CHEMBL1201607':...,Results: 200,Error: 598,Error: 598,Unknown: 503,Results: 200,Error: 598
ara-bte,No Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200,...,Results: 200 {'MONDO:0005301': 'True'},Error: 598,Error: 598,Results: 200 {'CHEMBL.COMPOUND:CHEMBL1201607':...,No Results: 200,Error: 598,Error: 598,Results: 200,No Results: 200,No Results: 200
ara-explanatory,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-improving,Results: 200 {'PUBCHEM.COMPOUND:2950270': 'True'},Results: 200,Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,...,No Results: 200,Error: 400,Error: 400,Error: 400,No Results: 200,"Results: 200 {'NCBIGene:120892': 'True', 'NCBI...","Results: 200 {'NCBIGene:3988': 'True', 'NCBIGe...",Results: 200,Results: 200,No Results: 200
ara-ncats,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,...,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503
ara-robokop,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,...,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500
ara-unsecret,Results: 200 {'PUBCHEM.COMPOUND:2950270': 'Fal...,No Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,...,Results: 200 {'MONDO:0005301': 'True'},No Results: 200,No Results: 200,No Results: 200,Error: 598,Error: 598,Error: 598,Results: 200,No Results: 200,No Results: 200
kp-cam,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200


<br>

### Highlight the cells In Excel

<br>

In [139]:
def highlight(v):
    if v.startswith('Results'):
        return 'background-color: %s' % 'green'
    elif v.startswith('Error'):
        return 'background-color: %s' % 'red'
    elif v.startswith('No Results'):
        return 'background-color: %s' % 'yellow'
    elif v.startswith('ARS Error'):
        return 'background-color: %s' % 'blue'
    elif v.startswith('Unknown'):
        return 'background-color: %s' % 'magneta'
    else:
        return

In [140]:
styled = df.style.applymap(highlight)

In [141]:
styled

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,A.Multiomics_BigGIM_DR_KP_EGFR,A.Multiomics_BigGIM_DR_KP_RHOBTB2,B.1_DILI-three-hop-interacts-with,B.2a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone,B.2b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol,B.2c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin,B.2d_DILI-fourth-one-hop-from-MESH_D003474_Curcumin,B.2e_DILI-fourth-one-hop-from-RXNORM_40068_DextranSulfate,B.2f_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone,B.2g_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5865_Prednisone,B.3_DILI_branched-four-hop,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule,C.2b_Etanercept_MultSclerosis_GeneSet_and_SmallMolecule,C.2c_Natalizumab_MultSclerosis_GeneSet_and_SmallMolecule,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
ara-aragorn,No Results: 200,Results: 200,Error: 598,No Results: 200,No Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,Error: 598,Error: 598,Results: 200,Results: 200,Results: 200,Error: 422,Results: 200,Error: 413,Error: 598,Results: 200 {'MONDO:0005301': 'True'},Results: 200 {'CHEBI:45783': 'False'},Results: 200 {'CHEBI:4875': 'False'},Error: 598,Results: 200,"Results: 200 {'NCBIGene:120892': 'True', 'NCBIGene:11315': 'True', 'NCBIGene:110357': 'False'}",Error: 598,Results: 200,Error: 598,Error: 598
ara-aragorn-exp,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404
ara-arax,Results: 200 {'PUBCHEM.COMPOUND:2950270': 'False'},Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Error: 598,Results: 200,Results: 200,Results: 200,Results: 200,Error: 400,Results: 200,Error: 598,Error: 598,Results: 200 {'MONDO:0005301': 'True'},Results: 200 {'CHEBI:45783': 'False'},Results: 200 {'CHEBI:4875': 'False'},Results: 200 {'CHEMBL.COMPOUND:CHEMBL1201607': 'False'},Results: 200,Error: 598,Error: 598,Unknown: 503,Results: 200,Error: 598
ara-bte,No Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Error: 502,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,Error: 598,Results: 200 {'MONDO:0005301': 'True'},Error: 598,Error: 598,Results: 200 {'CHEMBL.COMPOUND:CHEMBL1201607': 'True'},No Results: 200,Error: 598,Error: 598,Results: 200,No Results: 200,No Results: 200
ara-explanatory,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 400,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-improving,Results: 200 {'PUBCHEM.COMPOUND:2950270': 'True'},Results: 200,Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Error: 400,Results: 200,Results: 200,No Results: 200,No Results: 200,Error: 400,Error: 400

In [142]:
date = datetime.now().strftime("%Y_%m_%d-%I_%M_%S_%p")
wks_name = 'Workflow Progress Tracker_' + date

In [143]:
wks_name

'Workflow Progress Tracker_2021_10_01-01_33_39_PM'

In [144]:
styled.to_excel('/Users/priyash/Documents/GitHub/minihackathons/Notebooks/' + wks_name + '.xlsx')

#styled.to_excel(writer, sheet_name='progress_tracker')
#df2.to_excel(writer, sheet_name='edge_attribute')

/usr/local/lib/python3.9/site-packages/pandas/io/formats/excel.py:375: CSSWarning: Unhandled color format: 'magneta'
  warnings.warn(f"Unhandled color format: {repr(val)}", CSSWarning)


<br>

### Pushing dataframe to excel sheet on google drive

<br>


**Here I am using the google drive API to push the daatframe into an axcel sheet 
Every individula has the unique credential file that they need to create for google drive API -- 
"araxworkflowprogresstesting-2632632db8be.json" -- is the credential used from my drive. place this json file where
the ReadAndRunAllWorkFLows.ipynb will be. NB: i have removed my credntial file for privacy reasons. Always remove
the json file before making committs to the repo. To use googe Drive API follow: https://towardsdatascience.com/how-to-manage-files-in-google-drive-with-python-d26471d91ecd**

<br>

In [145]:
styled

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,A.Multiomics_BigGIM_DR_KP_EGFR,A.Multiomics_BigGIM_DR_KP_RHOBTB2,B.1_DILI-three-hop-interacts-with,B.2a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone,B.2b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol,B.2c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin,B.2d_DILI-fourth-one-hop-from-MESH_D003474_Curcumin,B.2e_DILI-fourth-one-hop-from-RXNORM_40068_DextranSulfate,B.2f_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone,B.2g_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5865_Prednisone,B.3_DILI_branched-four-hop,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule,C.2b_Etanercept_MultSclerosis_GeneSet_and_SmallMolecule,C.2c_Natalizumab_MultSclerosis_GeneSet_and_SmallMolecule,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
ara-aragorn,No Results: 200,Results: 200,Error: 598,No Results: 200,No Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,Error: 598,Error: 598,Results: 200,Results: 200,Results: 200,Error: 422,Results: 200,Error: 413,Error: 598,Results: 200 {'MONDO:0005301': 'True'},Results: 200 {'CHEBI:45783': 'False'},Results: 200 {'CHEBI:4875': 'False'},Error: 598,Results: 200,"Results: 200 {'NCBIGene:120892': 'True', 'NCBIGene:11315': 'True', 'NCBIGene:110357': 'False'}",Error: 598,Results: 200,Error: 598,Error: 598
ara-aragorn-exp,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404
ara-arax,Results: 200 {'PUBCHEM.COMPOUND:2950270': 'False'},Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Error: 598,Results: 200,Results: 200,Results: 200,Results: 200,Error: 400,Results: 200,Error: 598,Error: 598,Results: 200 {'MONDO:0005301': 'True'},Results: 200 {'CHEBI:45783': 'False'},Results: 200 {'CHEBI:4875': 'False'},Results: 200 {'CHEMBL.COMPOUND:CHEMBL1201607': 'False'},Results: 200,Error: 598,Error: 598,Unknown: 503,Results: 200,Error: 598
ara-bte,No Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Error: 502,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,Error: 598,Results: 200 {'MONDO:0005301': 'True'},Error: 598,Error: 598,Results: 200 {'CHEMBL.COMPOUND:CHEMBL1201607': 'True'},No Results: 200,Error: 598,Error: 598,Results: 200,No Results: 200,No Results: 200
ara-explanatory,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 400,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-improving,Results: 200 {'PUBCHEM.COMPOUND:2950270': 'True'},Results: 200,Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Error: 400,Results: 200,Results: 200,No Results: 200,No Results: 200,Error: 400,Error: 400

In [146]:
df['Query Type'] = 'Sync'

df = df[['Query Type']+ list(df.columns[:-1])]

In [147]:
#df.reset_index(inplace=True)

In [148]:
#df.rename({'index': 'ARA/KP'}, axis=1, inplace=True)

<br>

## Google Drive API

**Push the dataframe to a google sheet via google drive API and then format the google spread sheet to add hyperlink to pk's and color the cells

<br>

### Push Dataframe 1

<br>

In [149]:
import gspread
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import set_with_dataframe
from gspread_formatting import *

In [150]:
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    'araxworkflowprogresstesting-2632632db8be.json', scope)
gc = gspread.authorize(credentials)

In [151]:
spreadsheet_key = '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A'
#spreadsheet_key = '1sPpBIkxrHbQNiTm5oPs9-5KrjsyXcgaVAxknJj-u8pY'
#wks_name = 'Workflow Progress Tracker_' + date
d2g.upload(df, spreadsheet_key, wks_name, credentials=credentials, row_names=True)

APIError: {'code': 400, 'message': 'Invalid requests[0].addSheet: This action would increase the number of cells in the workbook above the limit of 5000000 cells.', 'status': 'INVALID_ARGUMENT'}

### Push Dataframe 2

<br>

In [64]:
wks2 = 'edge_attribute_source_' + date

In [65]:
spreadsheet_key = '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A'
#spreadsheet_key = '1sPpBIkxrHbQNiTm5oPs9-5KrjsyXcgaVAxknJj-u8pY'
#wks_name = 'Workflow Progress Tracker_' + date
d2g.upload(df2test, spreadsheet_key, wks2, credentials=credentials, row_names=True)

<Worksheet 'edge_attribute_source_2021_10_01-08_34_02_AM' id:1602560067>